# Data preparation utilities
> Functions that prepare label files and split the data into train validation and test sets

In [1]:
#hide
# default_exp dataprep_utils
from cellsegment.core import *
from nbdev.showdoc import *

In [2]:
# export
import random
from  cellsegment.core import *
import json
import shutil

import pandas
from fastai.vision import get_image_files, parallel, partial, PIL, Path, get_files
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage.exposure import histogram
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os

LABELS = [
        {'Name': 'Fluke, Liver', 'Code': '11', 'Fill': (255, 0, 0, 64)},
        {'Name': 'Fluke, Rumen', 'Code': '40', 'Fill': (0, 255, 0, 64)},
        {'Name': 'Other', 'Code': '', 'Fill': (255, 255, 0, 64)},
    ]

FILENAME_TRIM = False


In [4]:
# export

def createjson(fn='', offset=0):
    'create a Labelme compatable json file'
    data = {
        'version': '3.13.1',
        'flags': {},
        'shapes': [],
        'lineColor': [0, 255, 0, 255],
        'fillColor': [255, 255, 0, 20],
        'imagePath': fn,
        'offset': offset,
        'imageData': None,
        'imageHeight': 2464,
        'imageWidth': 2464}
    return data

see lableme at https://github.com/wkentaro/labelme/blob/master/README.md

In [5]:
# export

def annotate_json(data, shape_type='circle', points=None):
    # Annotate the json file with list of points
    assert shape_type == 'point' or shape_type == 'circle' or shape_type == 'rectangle'

    def add_anno(data, item):
        cx, cy = item['point']
        pnt_list = []
        if shape_type == 'point':
            pnt_list = [[cx, cy]]
        elif shape_type == 'rectangle':  # bounding points
            r = 40
            pnt_list = [[cx - r, cy - r], [cx + r, cy + r]]
        elif shape_type == 'circle':   # center & radius
            r = 20
            pnt_list = [[cx, cy], [cx + r, cy + r]]

        probability = str(item['probability']) if 'probability' in item else str(0)

        if item['label'] == 1:
            line_color = [255, 0, 0, 127]
            data['shapes'].append({
                "label": str(item['label']), "line_color": [255, 0, 0, 127], "fill_color": [255, 0, 0, 127],
                "points": pnt_list, "shape_type": shape_type, 'probability': probability
            })

        elif item['label'] == 2:
            line_color = [0, 255, 0, 127]
            data['shapes'].append({
                "label": str(item['label']), "line_color": [0, 255, 0, 127], "fill_color": [0, 255, 0, 127],
                "points": pnt_list, "shape_type": shape_type, 'probability': probability
            })

        else:
            line_color = [0, 0, 255, 127]
            data['shapes'].append({
                "label": str(item['label']), "line_color": [0, 0, 255, 127], "fill_color": [0, 0, 255, 127],
                "points": pnt_list, "shape_type": shape_type, 'probability': probability
            })

    def add_annotations(data, points):
        for item in points:
            add_anno(data, item)

    add_annotations(data, points)
    return data

In [6]:
# export

def __create_json_from_CSV(csv_fn, load_jpg_fn, offset=0, height=0, width=0):
    # Annotate the json file with predictions
    data = {
        'version': '3.13.1',
        'flags': {},
        'shapes': [],
        'lineColor': [0, 255, 0, 255],
        'fillColor': [255, 255, 0, 20],
        'imagePath': load_jpg_fn,
        'offset': offset,
        'imageData': None,
        'imageHeight': height,
        'imageWidth': width}

    # species_name = ['Nematodirus', 'Strongyle', 'other']

    egg_centers = []
    with open(csv_fn, mode='r') as f:
        line_count = 0
        for line in f:
            row, col, species = [int(x) for x in line.split(',')]
            egg_centers.append({"label": species, "particleType": species, "point": [row, col]})
            line_count += 1

    data = annotate_json(data, points=egg_centers)
    data
    return data

def csv_to_json_dir(src_path, dest_path, number_files='all'):
    """ Convert an entire directory of Techion CSV files to JSON files. Store in the dest directory
    CSV format is x,y,species_number
    1383,1571,2
    1687,1822,1
    2036,1327,1
    """
    print('Converting an entire directory of Techion CSV files to JSON files')
    fnames_csv = sorted(get_files(src_path, extensions=['.csv']))
    fnames_jpg = sorted(get_files(src_path, extensions=['.jpg']))
    if isinstance(number_files, int):
        fnames_csv = fnames_csv[:number_files]
        fnames_jpg = fnames_jpg[:number_files]
    print('Number of csv & jpg files to convert', len(fnames_csv), len(fnames_jpg))

    Path(dest_path).mkdir(parents=True, exist_ok=True)
    print(f"src_path {src_path}")
    print(f"dest_path {dest_path}")

    for i,fn in enumerate(fnames_csv):
        parent = Path(fn).parents[0]
        name = Path(fn).name.split('.')[0]
        jpg_fn = f"{parent}/{name}.jpg"
        json_fn = f"{parent}/{name}.json"

        if Path(jpg_fn).is_file():
            img = PIL.Image.open(jpg_fn)
            data = __create_json_from_CSV(fn, Path(jpg_fn).name, height=img.size[1], width=img.size[0])
            with open(json_fn, 'w') as outfile:
                # json.dump(data, outfile, indent=2)
                # print(f'Saving File {json_fn}')
                json.dump(data, outfile, ensure_ascii=False, indent=4)
                progress_bar(i+1,50)




In [7]:
# export

def crop2well(img, thres_adjust=1., bg_color=[0,0,0], op='crop'):
    '''Returns a square colour and gray image centered on the well with width same as the height.
    thres_adjust is  optional otsu threshold scaling multiplier
    Also returns the offset to the original image and the well region properties'''
    assert op=='mask' or op=='crop', 'op must be either "mask" or "crop"), the value was {}'.format(op)

    width = img.shape[0]
    img_gray = rgb2gray(img)
    thresh = threshold_otsu(img_gray)
    img_thresholded = closing(img_gray > thresh * thres_adjust, square(3))
    label_image = label(img_thresholded)

    well_region = None
    for region in regionprops(label_image):
        if region.area >= 100000:
            well_region = region
    # if region_center == 0: raise ValueError('No region_center found')

    r0, c0, r1, c1 = well_region.bbox
    if op=='mask':
        well_circle_mask = pad(well_region.convex_image, (img.shape[:2]), well_region.bbox[:2])
        img_crop = img[r0:r1, c0:c1]
        img_rgb = pad(img_crop, img.shape, well_region.bbox[:2], bg_color=bg_color)
        return img_rgb, img_thresholded, well_circle_mask, well_region
    elif op=='crop':
        well_center = (c0 + c1)//2
        offset = well_center - width//2
        img_rgb = img[:, offset:well_center + width//2, :]
        return img_rgb, offset

In [8]:
# export
def well_circle_mask(img, well_regionprops):
    return pad(well_regionprops.convex_image, (img.shape[:2]), well_regionprops.bbox[:2])

In [9]:
# export
def check_well_crop(img, well_regionprops):
    assert img.shape[0] == img.shape[1], "Expecting the cropped  well image to be square"
    bbox = well_regionprops.bbox
    bbox_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
    bbox_squareness = (bbox[2] - bbox[0]) / (bbox[3] - bbox[1])
    bbox_percent = bbox_area / (img.shape[0] * img.shape[0])
    max_well_area = 3.14156 * img.shape[0] * img.shape[0] / 4
    illum_well_percent = well_regionprops.area / max_well_area

    print('bbox_percent', bbox_percent)
    print('bbox_squareness', bbox_squareness)
    print('illum_well_percent', illum_well_percent)
    print('illum_well_saturation', illum_well_percent)


In [10]:
# export
def pad(im, shape, offset, bg_color=None):
    """
    array: Array to be padded
    reference_shape: tuple of size of ndarray to create
    offsets: list of offsets (number of elements must be equal to the dimension of the array)
    will throw a ValueError if offsets is too big and the reference_shape cannot handle the offsets
    """

    # Create an array of zeros with the reference shape
    if bg_color:
        result = np.ones(shape, im.dtype)
        result = result * np.array(bg_color)
    else:
        result = np.zeros(shape, im.dtype)
    # Create a list of slices from offset to offset + shape in each dimension
    insertHere = [slice(offset[dim], offset[dim] + im.shape[dim]) for dim in range(2)]
    # Insert the array in the result at the specified offsets
    result[insertHere] = im
    return result

In [11]:
# export

def get_image_size(fn):
    return PIL.Image.open(fn).size


def resize_crop2well_one(fn, i, src_path, dest_path, height):
    img = PIL.Image.open(src_path / fn.name)
    img_w, img_h = img.size
    scale = float(height / img_h)
    shape = (int(img_w * scale), height)
    img_cropped, offset = crop2well(np.asarray(img.resize(shape)))
    img_cropped = PIL.Image.fromarray(img_cropped.astype('uint8'), 'RGB')
    img_cropped.save(dest_path / f'{fn.stem}|{scale:.6f}|{offset:3}|.jpg', quality=90)

# def resize_crop2center_one(fn, i, src_path, dest_path, height):
#     img = PIL.Image.open(src_path / fn.name)
#     img_w, img_h = img.size
#     scale = float(height / img_h)
#     shape = (int(img_w * scale), height)
#     img_cropped, offset = crop2well(np.asarray(img.resize(shape)))
#     img_cropped = PIL.Image.fromarray(img_cropped.astype('uint8'), 'RGB')
#     img_cropped.save(dest_path / f'{fn.stem}|{scale:.6f}|{offset:3}|.jpg', quality=90)

In [12]:
# export
DEFAULT_LABEL_FILL = (255, 255, 255, 64)
def label_from_json(data, img, plot=False, radius=20):
    # data = json.load(open(load_json_fn(fn)))
    # img = PIL.Image.open(load_img_fn(fn))
    num_points = 0
    label_cnt_list = [{ 'Name': lab['Name'], 'Code': lab['Code'], 'Count': 0 } for lab in LABELS]
    for s, sh in enumerate(data['shapes']):
        fill = DEFAULT_LABEL_FILL
        for i, label in enumerate(LABELS):
            if sh["shape_type"] == "circle":
                if sh["label"] == label['Code']:
                    fill = label['Fill']
                    label_cnt_list[i]['Count'] += 1
                    num_points += 1
                    break

        # draw a circle at the circle
        if sh["shape_type"] == "circle":
            xy = np.asarray(sh["points"])
            # ave = np.mean(xy, axis=0)
            # cx, cy = ave[0], ave[1]
            cx, cy = xy[0][0], xy[0][1]
            draw = PIL.ImageDraw.Draw(img)

            draw.ellipse([cx - radius, cy - radius, cx + radius, cy + radius], outline=0, fill=fill)
            num_points += 1

    img = np.asarray(img)

    return img, num_points, label_cnt_list


In [13]:
# export
def create_one_label(fn, i, json_path, lab_path, colormap):
    data = json.load(open(f'{json_path}/{fn.stem}.json'))
    # img = PIL.Image.open(f'{src_path}/{fn.stem}.jpg')
    img = PIL.Image.new('RGB', (data["imageWidth"], data["imageHeight"]), color=(0, 0, 0))
    # h = data["imageHeight"]
    # w = data["imageWidth"]
    img, num_pnts, label_cnt_list = label_from_json(data, img, plot=False, radius=10)
    # ToDo: fix this code as it is not extendable
    lbl_pil_1 = (img[:, :, 0] >= 200) & (img[:, :, 1] <= 10) & (img[:, :, 2] <= 10)
    lbl_pil_2 = (img[:, :, 1] >= 200) & (img[:, :, 0] <= 10) & (img[:, :, 2] <= 10)
    lbl_pil = lbl_pil_1 + 2 * lbl_pil_2

    lbl_pil = PIL.Image.fromarray(lbl_pil.astype(np.uint8), mode='P')
    lbl_pil.putpalette(colormap)
    lbl_pil.save(f'{lab_path}/{fn.stem}.png')


In [14]:
# export
def resize_dir(file_data, src_path, dest_path, number_files='all', height=1200):
    """
    Resize  an entire directory. Store in the dest directory
    :param src_path:
    :param dest_path:
    :param number_files:
    :param height:
    :return:
    """

    df = pandas.read_csv(file_data, index_col=0)
    __number_files = df.shape[0]

    print(f'Number of image files: {__number_files}, Number to resize: {number_files}')
    Path(dest_path).mkdir(parents=True, exist_ok=True)

    if isinstance(number_files, int):
        __number_files = number_files

    for i in range(__number_files):
        f_stem = df.loc[i,'Name'].split('.')[0]
        scale = float(height) / df.loc[i,'Height']
        img = resize_file(f'{src_path}/{f_stem}.jpg', scale=scale)
        img.save(f'{dest_path}/{f_stem}.jpg', quality=90)

        progress_bar(i + 1, 50)
    return

In [15]:
# export
def resize_json_file(fn, scale=1, offset=0):
    data = json.load(open(fn))

    for s, sh in enumerate(data['shapes']):
        for p, pnt in enumerate(sh["points"]):
            data['shapes'][s]['points'][p][0] = round(data['shapes'][s]['points'][p][0] * scale)
            data['shapes'][s]['points'][p][1] = round(data['shapes'][s]['points'][p][1] * scale)
            data['shapes'][s]['points'][p][0] = data['shapes'][s]['points'][p][0] - offset
    data['imageData'] = None
    data['offset'] = offset
    data['scale'] = scale
    data['imageWidth'] = int (data['imageWidth'] * scale)
    data['imageHeight'] = int (data['imageHeight'] * scale)
    return data

In [16]:
# export
def resize_json_dir(file_data, src_path, dest_path, number_files='all', height=800):
    """
    Resize src_path directory of JSON files and store in dest_path directory
    There needs to be resized jpg files in the dest_path directory.
    An example is  226260 - 1|0.436047|221|.jpg  where |0.436047|221| marks the factor and offset factors to apply
    :param src_path: Source path where json files are
    :param dest_path: Destination path to store resized json files
    :param number_files: Number of json files to process, leave empty for all files in directory
    :return: number of file processed
    """
    df = pandas.read_csv(file_data, index_col=0)
    __number_files = df.shape[0]

    print(f'Number of JSON files: {__number_files}, Number to resize: {number_files}')
    Path(dest_path).mkdir(parents=True, exist_ok=True)
    if isinstance(number_files, int):
        __number_files = number_files

    for i in range(__number_files):
        f_stem = df.loc[i,'Name'].split('.')[0]
        scale = float(height) / df.loc[i,'Height']
        offset = 0
        data = resize_json_file(f'{src_path}/{f_stem}.json', scale=float(scale), offset=int(offset))
        data['imagePath'] = df.loc[i,'Name']
        with open(f'{dest_path}/{f_stem}.json', 'w') as outfile:
            json.dump(data, outfile, ensure_ascii=False, indent=4)

        progress_bar(i + 1, 50)
    print('')
    print(i+1, ' json files processed')
    return i+1

In [17]:
# export
def shuffle_csv(file_csv,random_state=None):
    df = pandas.read_csv(file_csv, index_col=0)
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    df.to_csv(file_csv, index=False)

In [18]:
# export
def split_filenames(file_csv, num_train=0.70, num_val=0.15):
    """
    shuffle file names, split into train valid and test and update file_data.csv with labels

    :param file_csv: csv file with file names and data
    :param num_train:
    :param num_val:
    :param seed:
    :return:
    """
    df = pandas.read_csv(file_csv, index_col=0)
    __number_files = df.shape[0]

    # add Op column
    operation = ['Train'] * int(__number_files * num_train)
    operation = operation + ['Valid'] * int(__number_files * num_val)
    operation = operation + ['Test'] * (__number_files - len(operation))

    df['Op'] = operation
    df.to_csv(file_csv, index=False)

    # train_df = df.loc[df['Op'] == 'Train']
    # valid_df = df.loc[df['Op'] == 'Valid']
    # test_df = df.loc[df['Op'] == 'Test']
    # trainfiles = train_df['Name'].tolist()
    # validfiles = valid_df['Name'].tolist()
    # testfiles = test_df['Name'].tolist()
    #
    # assert len(trainfiles) + len(validfiles) + len(testfiles) == __number_files, 'The number of train, val and test files do not match the original number'
    # print(
    #     f'Number of train, val and test files ({len(trainfiles)}, {len(validfiles)}, {len(testfiles)}) == {__number_files}')
    #
    # return trainfiles, validfiles, testfiles

    return None

In [19]:
# export

def __crop_image(jsonfn, imgfn, dest_path, size=200, op:str='', debug=False):
    """
    Crop an image into multiple sub-tiles centered on each egg
    :param jsonfn: JSON file with egg centers
    :param imgfn: image to be cropped
    :param dest_path: directory to put cropped tiles into
    :param size: pixel size of crop
    :param filename_trim: remove the scale and offset portions of filename and remove extra spaces, 226260 - 1-0.436047-221-.jpg ->  226260-1.jpg
    :return: number of cropped images
    """
    img = np.asarray(PIL.Image.open(imgfn))
    data = json.load(open(jsonfn))
    suffix = Path(imgfn).suffix
    dest_path = Path(dest_path)
    imgfn = Path(imgfn).stem
    assert suffix == '.jpg' or suffix == '.png', "image file type must be jpg or png"
    imgShape = img.shape

    colormap = colormap_segmentation_labels()
    n = 0
    croplist = []
    for n, sh in enumerate(data['shapes']):
        if sh["shape_type"] == "circle":
            xy = np.asarray(sh["points"])
            ave = np.mean(xy, axis=0)

            cx = min(int(round(ave[1])), imgShape[0] - size // 2)
            cx = max(cx, size // 2)
            cy = min(int(round(ave[0])), imgShape[1] - size // 2)
            cy = max(cy, size // 2)

            minr = cx - size // 2; maxr = cx + size // 2
            minc = cy - size // 2; maxc = cy + size // 2

            crop = img[minr:maxr, minc:maxc]

            # trim offset and scale from filename
            if FILENAME_TRIM:
                a, scale, offset, b = imgfn.split('|')
                savefn = f'{a}'.replace(" ", "")
            else:
                savefn = imgfn

            savefn = f'{savefn}-{n}{suffix}'
            # savefn = f'{dest_path}/{savefn}-{n}{suffix}'

            # print('saving', savefn)
            if not debug:
                if suffix == '.jpg':
                    PIL.Image.fromarray(crop.astype(np.uint8)).save(dest_path/savefn, quality=90)
                elif suffix == '.png':
                    lbl_pil = PIL.Image.fromarray(crop.astype(np.uint8), mode='P')
                    lbl_pil.putpalette(colormap)
                    lbl_pil.save(dest_path/savefn)
            else:
                print('x', end='')


            item = {'Name': savefn, 'Label': sh["label"], 'Op': op}
            croplist.append(item)
    # print(f'Saved {n} files in {dest_path}')
    return croplist

In [20]:
# export
def crop_img_dir(file_data:str, json_path:str, src_path:str, dest_path:str, number_files='all', DEBUG=False):
    """
    Crop directory of image files based on json centers and store in dest directory
    - `file_data:`    file_data.csv  
    - `src_path:`     path where json files and image files to be cropped are  
    - `dest_path:`    Destination path to store cropped files  
    - `number_files:` Number of json files to process, leave empty for all files in directory  
    - `size:`         pixel size of crop  
    - `return:`       total number of cropped images`  
    """
    if DEBUG: print (f'Debug = {DEBUG} so not saving files')
    # assert subdir in ['Train', 'Val', 'Test', None], "subdir must one of 'Train', 'Val', 'Test' or None"

    src_df = pandas.read_csv(file_data, index_col=0)
    if number_files == 'all':
        __number_files = src_df.shape[0]
    elif isinstance(number_files, int) :
        __number_files = number_files
    else:
        __number_files = 0

    json_path = Path(json_path)
    src_path = Path(src_path)
    dest_path = Path(dest_path)

    (dest_path/'Train').mkdir(parents=True, exist_ok=True)
    (dest_path/'Test').mkdir(parents=True, exist_ok=True)
    (dest_path/'Label').mkdir(parents=True, exist_ok=True)
    (dest_path/'Error').mkdir(parents=True, exist_ok=True)

    print(f'{__number_files} files to process in {src_path}' )

    misslist = []
    croplist = []
    vallist = []
    for i in range(__number_files):
        imgfn = src_df.loc[i,'Name']
        f_stem = imgfn.split('.')[0]
        labfn = f'{f_stem}.png'
        jsonfn = f'{f_stem}.json'
        if (src_path/imgfn).exists():
            fn = imgfn
            file_type = "IMAGE"
        elif (src_path/labfn).exists():
            fn = labfn
            file_type = "LABEL"
        else:
            fn = None
        if (json_path/jsonfn).exists() and fn is not None:
            op = src_df.loc[i, 'Op']
            if file_type == "LABEL":
                cropdir = dest_path / 'Label'
            elif op == 'Valid' or src_df.loc[i, 'Op'] == 'Train':
                cropdir = dest_path / 'Train'
            elif op == 'Test':
                cropdir = dest_path / 'Test'
            else:
                print(f'File: {imgfn} has no Op label or is not a label file')
                cropdir = dest_path / 'Error'

            # if op == 'Valid' and file_type == "IMAGE":
            #     vallist += lst

            lst = __crop_image(json_path/jsonfn, src_path/fn, cropdir, op=op, size=200, debug=DEBUG)

            croplist += lst


            progress_bar(i + 1, 50)
        else:
            misslist.append(f_stem)

    print('')
    print(f'Missed {len(misslist)} and  Cropped {len(croplist)} files in {dest_path}')

    # savefn = dest_path / 'valid.txt'
    # print(f"Saving {len(vallist)} valid file names in {savefn}")
    # if len(vallist) > 0:
    #     with open(savefn, "w") as a:
    #         for item in vallist:
    #             fn = item['Name']
    #             if Path(fn).suffix == '.jpg' :
    #                 a.write(fn + os.linesep)

    return misslist, croplist

In [21]:
# export
def move_files_to_dir( movefiles, srcpath, destpath, operation='move', extns=['.jpg','.json','.png']):
    """
    Move random shuffle of the source directory to the Train, Val and Test directories

    :param movefiles: list of files to move
    :param srcpath: path where to src train files
    :param destpath: path where to put test files
    :param extns: list of extensions to try
    :return: cnt of files_moved, files_missed
    """

    files_moved = [0]*3
    files_missed = [0]*3
    Path(destpath).mkdir(parents=True, exist_ok=True)
    for fn in movefiles:
        fn = Path(f'{srcpath}/{fn}')
        for i, ext in enumerate(extns):
            if fn.with_suffix(ext).exists():
                if operation == 'copy':
                    shutil.copy(str(fn.with_suffix(ext)), str(destpath))
                elif operation == 'move':
                    shutil.move(str(fn.with_suffix(ext)), str(destpath))
                files_moved[i] += 1
            else:
                files_missed[i] += 1

    print( "Files Moved", files_moved)
    print( "Files Missed", files_missed)

        # print(f'Moved {len(jpg_cnt)} jpg files, {json_cnt} json files and  {png_cnt} png files to {movefiles}')

    return files_moved, files_missed

In [22]:
# export
def old_move_files_to_dir( movefiles, srcpath, destpath, operation='move', extns=['.jpg','.json','.png']):
    """
    Move random shuffle of the source directory to the Train, Val and Test directories

    :param movefiles: list of files to move
    :param srcpath: path where to src train files
    :param destpath: path where to put test files
    :param extns: list of extensions to try
    :return: cnt of files_moved, files_missed
    """

    files_moved = [0]*3
    files_missed = [0]*3
    Path(destpath).mkdir(parents=True, exist_ok=True)
    for fn in movefiles:
        fn = Path(f'{srcpath}/{fn}')
        for i, ext in enumerate(extns):
            if fn.with_suffix(ext).exists():
                if operation == 'copy':
                    shutil.copy(str(fn.with_suffix(ext)), str(destpath))
                elif operation == 'move':
                    shutil.move(str(fn.with_suffix(ext)), str(destpath))
                files_moved[i] += 1
            else:
                files_missed[i] += 1

    print( "Files Moved", files_moved)
    print( "Files Missed", files_missed)

        # print(f'Moved {len(jpg_cnt)} jpg files, {json_cnt} json files and  {png_cnt} png files to {movefiles}')

    return files_moved, files_missed

In [23]:
# export
def create_labels_dir(json_path, dest_path, number_files='all'):
    """
     Create label png images based on CSV files and store in dest directory

    :param json_path: Source path where json files are
    :param dest_path: Destination path to store created png label files
    :param number_files: Number of json files to process, leave empty for all files in directory
    :return: nil

    """
    
    lab_path = Path(dest_path)
    lab_path.mkdir(parents=True, exist_ok=True)

    fnames_json = sorted(get_files(json_path, extensions=['.json']))
    if isinstance(number_files, int):
        fnames_json = fnames_json[:number_files]
    colormap = colormap_segmentation_labels()
    print("Number of json files to process", len(fnames_json))
    
    if 1:
        for i,fn in enumerate(fnames_json):
            create_one_label(fn, i, json_path, lab_path, colormap)
            progress_bar(i + 1, 50)
    else:
        parallel(partial(create_one_label, json_path=json_path, lab_path=lab_path, colormap=colormap), fnames_json, leave=True)
    print("Number of labels files created", len(fnames_json))


